In [1]:
# import external libraries
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import re

import matplotlib.pyplot as plt
import seaborn as sns

**The following datasets are sourced from these websites, unless otherwise stated:**

[Swiss Impex](https://www.gate.ezv.admin.ch/swissimpex/index.xhtml) for data on food imports

[Swiss Farmers Union](https://www.sbv-usp.ch/fr/service/statistiques/) for various agricultural data

### Main question driving the project: 
> ## How can a Swiss consumer best adapt their diet to reduce their carbon "foodprint", based on all the factors that go into the carbon emissions of food production?

As is slowly becoming a widely-known fact, the meat and animal product industry is significantly more carbon-intensive than plant-based foods. The amount of CO2 produced per kg (or even kcal) of meat/animal product is much higher than the amount of CO2 for the same mass/energy equivalent of plants, e.g., fruits, vegetables, legumes. ([Center for Sustainable Systems, 2018](http://css.umich.edu/factsheets/carbon-footprint-factsheet)) One might think, then, that the best way to reduce one's "carbon foodprint" (carbon footprint due to food production) is to take on a vegetarian or vegan diet.

What is intriguing is that this may not be the answer in all cases. Based on where food is sourced, what time of year it is grown, and how it is produced, the carbon emissions can change drastically. A hot house tomato grown out of season can produce less carbon than pasture-fed goat meat, for example. 

An important question that data can answer, then, is how an eco-friendly citizen can best adapt their diet, based on their specific living circumstances, to reduce their carbon foodprint. Here, we will perform an in-depth analysis on Switzerland, providing a consumer with the knowledge necessary to produce less carbon emissions due to their diet. This will include an analysis of carbon intensities by food type and the various types and sources of food that are imported into Switzerland. After all, since Switzerland has a fairly dense population for its small size, it imports a significant amount of its food:

"According to the Federal Agriculture Office, Switzerland’s gross self-sufficiency rate in 2015 was 59%. The degree of self-sufficiency is defined as the ratio of domestic production to total domestic consumption. With imported animal feed taken into account, the net level of self-sufficiency was 51% that year.

But a closer look at the data reveals major differences across products. The country has been able to produce almost 100% of its animal foodstuffs for years, yet has managed only about 40% self-sufficiency for plant-based foods, a rate that fluctuates year on year as harvests are highly weather-dependent." (From swissinfo.ch)

Depending on where and what types of food Switzerland is importing its non-meat food products from, this raises the possibility that a Swiss citizen might have a smaller carbon foodprint by eating more meat, contradicting the typical recommendation. Would living with a vegan diet require an excessive amount of imported foods, outweighting the carbon offset of not eating meat? Or is there a large enough amount of carbon produced by the meat industry that eating a plant-based diet is still more sustainable? 

Let's start by looking at Swiss food imports on a macro scale.

In [ ]:
# this sheet is food, beverages, and tobacco
imported_goods = pd.read_excel('../data/nature-of-goods-imports.xlsx', skiprows=5, sheet_name='01.1')
imported_goods.dropna(how='all',inplace=True)
imported_goods.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True) #empty columns
imported_goods.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
imported_goods['Commercial Partner'] = imported_goods['Commercial Partner'].str.strip() # get rid of leading whitespace
imported_goods.head()

Let's look at how much of the food, beverages, and tobacco that Switzerland imports comes from its nearest neighboring countries. The farther a country is, the more carbon emissions it would cost to import that food.

In [ ]:
# I just hand-wrote this list, it doesn't actually mean anything. Just picked nearby countries
nearby_countries = ['Portugal', 'Spain', 'France', 'Germany', 'Italy', 'Austria', 'Belgium', 'Netherlands', 'Czech Republic', 'Slovenia', 'Croatia']
imported_goods_nearby = imported_goods[imported_goods['Commercial Partner'].isin(nearby_countries)].copy()
imported_goods_nearby.head()

In [ ]:
total_nearby_imported = imported_goods_nearby['Quantity (kg)'].sum()
100*total_nearby_imported/imported_goods.iloc[0][2]

Approximately 74% of Switzerland's imported food, beverage, and tobacco products come from fairly nearby countries. That's pretty good! It means imports would have a lower carbon output than if Switzerland imported a lot from far away. (It does not, however, indicate anything to us about whether the countries that Switzerland sources its imports from are carbon efficient. For example, Germany might have higher carbon intensities per kilo of wheat produced than the same product in France.) 

What about Switzerland's imported feedstuffs for animals? (If the Swiss import a lot of feed, that contributes to an even higher carbon intensity for raising meat)

In [ ]:
# this sheet is feeding stuffs for animals
imported_feed = pd.read_excel('../data/nature-of-goods-imports.xlsx', skiprows=5, sheet_name='01.2')
imported_feed.dropna(how='all',inplace=True)
imported_feed.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True) #empty columns
imported_feed.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
imported_feed['Commercial Partner'] = imported_feed['Commercial Partner'].str.strip() # get rid of leading whitespace
imported_feed.head()

In [ ]:
imported_feed_nearby = imported_feed[imported_feed['Commercial Partner'].isin(nearby_countries)].copy()
total_nearby_imported_feed = imported_feed_nearby['Quantity (kg)'].sum()
100*total_nearby_imported_feed/imported_feed.iloc[0][2]

Nearly three-quarters of imported animal feed is imported from nearby countries, which means when Switzerland must import feed, they mostly minimize carbon emissions from the process by reducing the travel distance of the feed. More importantly, a report from the State Secretariat for Economic Affairs ("Concentrate Animal Feed as an Input Good in Swiss Agricultural Production - The Effects of Border Protection and Other Support Measures") claims 90% of animal feed used in Switzerland is domestically produced, drastically reducing the carbon impact of the meat industry in the country.

This percentage can be found in this PDF document (see the table on page 8):

https://www.sbv-usp.ch/fileadmin/sbvuspch/00_Bilder/06_Services/Agristat/Statistiken/Produktionsmittel__Umwelt/SES2018_Kap04_Produktionsmittel-Umwelt.pdf

In [ ]:
# the following image is from the Concentrate Animal Feed report
# it gives a good idea of what we should see in the data about imported/domestically produced feed
from IPython.display import Image
Image('swiss_feed_by_origin.png', width=800,height=600)

We will need to pay attention to where food is coming from by animal type. For example, 90% of cow and cattle feed is domestically produced, whereas that number is only 52% for pork and 30% for poultry (found in the Concentrate Animal Feed report above, where it was sourced from the PDF linked above). So even if chickens are less carbon intensive, they require more imported feed which might increase their carbon output!

Let's now look at how much meat Switzerland imports.

In [ ]:
# should download this data for 2017 instead to match swiss emissions and consumption data
# "selected" meat imports since there is way more detail available, I just didn't download it
meat_imported = pd.read_excel('../data/selected-meat-imports.xlsx', skiprows=5, sheet_name='02')
meat_imported.dropna(how='all',inplace=True)
meat_imported.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True)
meat_imported.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
meat_imported.head()

As we can see from the above table, Switzerland imported 77 million kg of meat and edible offal in 2018 (equivalent to 77 thousand tonnes). The breakdowns by country are also provided.

Now let's look at what Switzerland's population _consumes_. 

In [ ]:
# Note: all of these values are from 2017 (earlier years are also available)
food_consumption_types = pd.read_excel(r'../data/food_consumption_by_type_of_food.xlsx', skiprows=8)
food_consumption_types.dropna(inplace=True)
food_consumption_types.rename({'1000 t':'Quantity, total, 1000 tonnes', 'kg ':'Quantity, kg/person/year', ' t':'Protein, total, tonnes', 'g':'Protein, g/person/day', '%':'Protein, % indigenous production', 'TJ':'Energy intake, total, TJ', 'KJ':'Energy intake, kj/person/day', '%.1':'Energy intake, % indigenous production'}, inplace=True, axis=1)
food_consumption_types.head()


Specifically, let's look at meat from the above table. How much meat did Swiss people consume in 2017?

In [ ]:
food_consumption_types.iloc[17]

As can be seen, the Swiss consumed 413 thousand tonnes, but they imported 77 thousand tonnes. This means most of the Swiss meat is produced within the country. (This calculates to be 81% of consumed Swiss meat to be domestically produced/non-imported)

Some other important facts, as provided by an article on swissinfo.ch: Switzerland produces more than enough milk and milk products to satisfy its needs, but has to import the majority of its fruits, vegetables, and nuts. Animal feed is one of the major imports, as well as staples like rice and wheat.

##### How does this affect emissions?
The next thing we can look at is how intensive (in terms of carbon emissions) it is for Switzerland to produce certain types of food.

In [ ]:
# this data is from FAO, emissions intensities
swiss_emis = pd.read_csv(r'../data/swiss_emissions_intensities.csv')
swiss_emis = swiss_emis[swiss_emis['Unit'] == 'kg CO2eq/kg product']
swiss_emis

# Where Brooke sees this going from here
Next, we'd have to look at what types of food Switzerland typically imports from other countries, then look at the emissions intensities for those foods in those countries. See notes in the last cell about the typical Swiss diet.

I don't know how we're going to approach calculating emissions for imported animals and meat (meaning, also including the carbon emissions of transport). I haven't thought about/researched that yet.

The data from Swiss-Impex (linked in the top of the notebook) is extremely detailed and has a lot more information to give. Feel free to check it out and get a sense of other data that we could use from there.

Points of discussion, which we can mention/speculate on but which our analysis will _not_ be able to address:

* Animals often are fed with by-products of human food production. If people eat less meat/animal products, we will have more food waste to dispose of
* We will probably only look at CO2, which is only one of the greenhouse gases

Perhaps we don't need a dataset about Swiss eating habits? (menuCH study data is restricted)
We should probably just look at carbon emissions of the normal recommended diet
Then we can pull some metadata of menuCH (Swiss consumption habits) from a report I found (e.g. "The Swiss population consumes an average of 220 g milk and dairy products such as cheese and yoghurt per person per day, which equates to 2 portions. They eat 150 g of other protein-based food such as meat, fish and tofu per day, of which 110 g comes from meat and meat products alone.")

Advice from Akhil (TA): At the very beginning, state all assumptions and justify them (one major assumption: we are assuming the given situation in Swiss agriculture and economics. We are NOT considering what the government can do (e.g. altering crops produced for exports, etc). Only what a consumer can do in the given situation)

Assumptions:
* There is no food waste
* Global averages used for all countries carbon intensities by type of crop
* Mismatched datasets, so we will assume that data coming from FAO and Impex is roughly correct

Other thoughts:

We can tie in land use by this connection: imports of feed have been increasing over the past few years because of reduction in domestic production, due to expansion of grass cultivation on arable land, which was due to lack of economic attractiveness for growing feed cereals (prices of cereals for human consumption are much higher, so more profitable)

Seasonality is super important! For example, locally harvested apples that have been in storage (due to not being in season) for 10 months are more carbon intensive, due to the energy to keep them fresh in storage, than fresh apples harvested and shipped from very far away

Food miles: let's start with just a fixed average emission rate per km, then build up to ships vs planes, then maybe look at seasonality

In [ ]:
# starting somewhat anew
# let's look at fruits. According to the consumption dataset from the Federal Office of Statistics:
food_consumption_types.iloc[10]

In [ ]:
def clean_impex_dataset(file_location, sheet):
    df = pd.read_excel(file_location, skiprows=5, sheet_name=sheet)
    df.dropna(how='all',inplace=True)
    df.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], inplace=True) #empty columns
    df.rename({'Unnamed: 0':'Year', 'Unnamed: 1':'Commercial Partner'}, inplace=True, axis=1)
    df['Commercial Partner'] = df['Commercial Partner'].str.strip() # get rid of leading whitespace
    return df

In [ ]:
# 18.5% of fruits consumed are domestically produced. Let's now see how much fruit is
# imported, so we can calculate domestic production vs imports
# The following data all comes from Swiss Impex
bananas_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0803')
exotic_fruit_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0804')
citrus_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0805')
grapes_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0806')
melons_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0807')
apples_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0808')
stone_fruit_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0809')
berries_imports = clean_impex_dataset('../data/fruits_and_veggies_imported.xlsx', '0810')

In [ ]:
# take the sum of total trade for each type of fruit
kg_fruit_imported = bananas_imports.iloc[0][2]+exotic_fruit_imports.iloc[0][2]+citrus_imports.iloc[0][2]+\
grapes_imports.iloc[0][2]+melons_imports.iloc[0][2]+apples_imports.iloc[0][2]+stone_fruit_imports.iloc[0][2]+\
berries_imports.iloc[0][2]
kg_fruit_imported

In [ ]:
# If this makes up 81.5% of consumed fruit in Switzerland, then Switzerland consumes:
kg_fruit_imported/(.815*8500000)

kg of fruit per person per year

This does not match with the given 115 kg of fruit per person per year from the dataset that provided the 18.5% domestic production number, but it is about on par with the bookmarked page 9 of the report, which says Swiss people eat about 83 kg of fruit per year per person. So let's go with it?

Now let's look at the carbon intensity of producing fruit in Switzerland, vs the carbon intensity of each of its trading partners, specific to their types of fruit.

... Ok, so it turns out estimates of carbon intensities vary WIDELY. The production method of just one type of fruit in one country can vary. For example, bananas are somewhere between 300 and 1200 kg CO2e per kg of bananas. Plus, how fruit is transported makes a large difference. Bananas are usually shipped by boat since they last for a long time, but fresher fruit (e.g. berries?) might be flown, which is much more carbon expensive. And apparently carbon intensities may already include the carbon cost of transport? Not sure how that's possible since it depends where the item ships to

In [ ]:
# let's look at FAO data on the amount of fruits and veggies Switzerland grows/produces
swiss_fruit_veg_prod = pd.read_csv(r'../data/swiss_fruit_veggie_production_FAO.csv')
swiss_fruit_veg_prod.head()

Maybe we _will_ have to look at exports, since what is grown is not necessarily all available for domestic consumption.

Goals for the upcoming milestone: focus on Switzerland
* Among the various datasets which give the same data but different numbers, we need to pick which one(s) we will use and justify it. This includes: percentage of domestic production by food group; amount by food group eaten by Swiss people--use FAO and Impex data, NOT federal statistics office
* Figure out the emissions intensities by food group for Switzerland--will just use the global average for Switzerland, can copy-paste it from the article here: https://www.sciencedirect.com/science/article/pii/S0959652616303584
* How much does Switzerland consume? -- calculate from imports, exports, and domestic production
* How much does Switzerland produce vs. supply its needs with imports?
* Update the README

Goals to address _after_ this milestone:
* Figure out the percentage for each food type which is transported by each method (for example, 40% of bananas are transported by ship, 30% by rail, etc.)
* Calculate the distances between each country and Switzerland
* Incorporate global emissions averages data by food group (assume the global average for every country except Switzerland): see page 11 of this report: https://core.ac.uk/download/pdf/76958641.pdf
* Combine these two datasets to produce the average carbon emission intensity for imported foods by food type

Ideas from TAs for how to further expand our project (and maybe incorporate machine learning or other related techniques):

* What is good for the environment? What is healthy? People want varied diets too! 
* It's ok if your data processing is not heavy, as long as your data story is beautiful and well thought-out
* Could maybe incorporate nutrition data, and optimize diet based on carbon and nutrition tradeoffs (could even define an index of the ratio of healthiness to carbon impact for each food, and then rank food types)
* Can also add things like clustering just to support our conclusions
* Can look at other countries' footprints too!